# Test des Intent Blocks:

In [1]:
from transformers import pipeline
import os
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset 

In [2]:
df = pd.read_csv('data/intent_data.csv')
df.head()

,text,label
0,Wie spät ist es?,question
1,Wer ist der aktuelle Bundespräsident?,question
2,Was ist die Hauptstadt von Australien?,question
3,Erzähl mir etwas über die Geschichte der Raumf...,question
4,Wie funktioniert Photosynthese?,question


In [3]:
df.dtypes

text     object
label    object
dtype: object

In [4]:
np.shape(df)

(199, 2)

In [5]:
label_counts = df['label'].value_counts()
print("Häufigkeit der Labels im Datensatz:")
print(label_counts)

Häufigkeit der Labels im Datensatz:
label
question           50
summary_request    50
internet_search    50
feedback           49
Name: count, dtype: int64


Fine Tuning DistilBERT:

In [6]:
# Labels numerisch kodieren und Zuordnungen erstellen
# Dies ist wichtig, da das Modell mit numerischen IDs arbeitet
unique_labels = df['label'].unique()
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for i, label in enumerate(unique_labels)}

df['label_id'] = df['label'].map(label_to_id)

print("\nLabel-Zuordnungen:")
print("String zu ID:", label_to_id)
print("ID zu String:", id_to_label)

# Daten in Trainings- und Testsets aufteilen
# 'stratify=df['label']' sorgt dafür, dass die Verteilung der Labels in beiden Sets gleich ist
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Pandas DataFrames in Hugging Face Datasets konvertieren
# 'drop(columns=['__index_level_0__'], errors='ignore')' ist notwendig, da Pandas einen temporären Index hinzufügen kann
train_dataset = Dataset.from_pandas(train_df.drop(columns=['__index_level_0__'], errors='ignore'))
test_dataset = Dataset.from_pandas(test_df.drop(columns=['__index_level_0__'], errors='ignore'))

print(f"\nTrainingsdaten: {len(train_dataset)} Beispiele")
print(f"Testdaten: {len(test_dataset)} Beispiele")

# --- 2. Tokenizer und Modell laden ---
print("\nSchritt 2: Tokenizer und Modell laden...")
# Wir verwenden ein multilinguales, cased (Groß-/Kleinschreibung beachtendes) DistilBERT Modell
# Es ist gut für Deutsch geeignet und kleiner/schneller als das volle BERT.
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# AutoModelForSequenceClassification lädt ein vortrainiertes Modell
# und fügt automatisch einen Klassifikations-Layer für unsere 'num_labels' hinzu.
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels), # Anzahl der Intents
    id2label=id_to_label,          # Mapping von ID zu Label-String
    label2id=label_to_id           # Mapping von Label-String zu ID
)
print(f"Modell '{model_name}' und Tokenizer geladen.")


# --- 3. Daten tokenisieren ---
print("\nSchritt 3: Daten tokenisieren...")
def tokenize_function(examples):
    # 'truncation=True' schneidet längere Texte ab, damit sie zur maximalen Länge des Modells passen
    # 'padding=True' fügt kürzere Texte mit Füllzeichen auf die maximale Länge auf
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Spalte 'label_id' in 'labels' umbenennen, da der Hugging Face Trainer 'labels' erwartet
tokenized_train_dataset = tokenized_train_dataset.rename_column("label_id", "labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("label_id", "labels")

# Ursprüngliche 'text' und 'label' Spalten entfernen, da das Modell die tokenisierten IDs benötigt
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text", "label"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text", "label"])

# Stellen Sie sicher, dass die Labels vom Typ Long sind (wichtig für PyTorch/TensorFlow)
tokenized_train_dataset.set_format("torch")
tokenized_test_dataset.set_format("torch")

print("Daten tokenisiert und formatiert.")


# --- 4. Metriken-Funktion definieren ---
# Diese Funktion wird verwendet, um die Leistung des Modells während des Trainings zu bewerten
print("\nSchritt 4: Metriken-Funktion definieren...")
def compute_metrics(eval_pred):
    logits, labels = eval_pred # Logits sind die Rohergebnisse des Modells, Labels sind die wahren Klassen
    predictions = np.argmax(logits, axis=-1) # Die Klasse mit dem höchsten Logit ist die Vorhersage

    # Berechnung von Precision, Recall und F1-Score
    # 'average='weighted'' berücksichtigt die Ungleichheit der Klassenverteilung
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0)
    acc = accuracy_score(labels, predictions) # Genauigkeit

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
print("Metriken-Funktion definiert.")


# --- 5. Training konfigurieren und starten ---
print("\nSchritt 5: Training konfigurieren und starten...")
output_dir = "./my_german_intent_analyzer" # Verzeichnis, in dem das Modell gespeichert wird

training_args = TrainingArguments(
    output_dir=output_dir,
				save_strategy="epoch",              # Ausgabe-Verzeichnis für Checkpoints und Ergebnisse
    eval_strategy="epoch",              # Bewertung nach jeder Epoche
    learning_rate=2e-5,                 # Typische Lernrate für Fine-Tuning von Transformers
    per_device_train_batch_size=8,      # Batch-Größe pro Gerät für Training
    per_device_eval_batch_size=8,       # Batch-Größe pro Gerät für Bewertung
    num_train_epochs=3,                 # Anzahl der Trainings-Epochen
    weight_decay=0.01,                  # Regularisierung zur Vermeidung von Overfitting
    save_total_limit=2,                 # Nur die 2 besten Checkpoints speichern
    load_best_model_at_end=True,        # Bestes Modell (basierend auf eval_metric) am Ende laden
    metric_for_best_model="f1",         # Metrik, die zur Bestimmung des "besten" Modells verwendet wird
    greater_is_better=True,             # Höherer F1-Score ist besser
    logging_dir='./logs',               # Verzeichnis für TensorBoard-Logs
    logging_steps=10,                   # Logs alle 10 Schritte ausgeben
    report_to="none"                    # Keine Berichte an Online-Dienste wie wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer, # Tokenizer auch an den Trainer übergeben
    compute_metrics=compute_metrics
)

# Starten des Trainings
print("Training startet...")
trainer.train()
print("Training abgeschlossen.")


# --- 6. Modell speichern ---
# Das beste Modell (gemäß 'load_best_model_at_end=True') ist bereits im 'model'-Objekt geladen.
# Wir speichern es nun im angegebenen Output-Verzeichnis.
print(f"\nSchritt 6: Modell speichern in '{output_dir}'...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir) # Auch den Tokenizer speichern!
print("Modell und Tokenizer erfolgreich gespeichert.")



Label-Zuordnungen:
String zu ID: {'question': 0, 'summary_request': 1, 'feedback': 2, 'internet_search': 3}
ID zu String: {0: 'question', 1: 'summary_request', 2: 'feedback', 3: 'internet_search'}

Trainingsdaten: 159 Beispiele
Testdaten: 40 Beispiele

Schritt 2: Tokenizer und Modell laden...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modell 'distilbert-base-multilingual-cased' und Tokenizer geladen.

Schritt 3: Daten tokenisieren...


Map:   0%|          | 0/159 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

C:\Users\mauro\AppData\Local\Temp\ipykernel_17552\2249414426.py:111: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Daten tokenisiert und formatiert.

Schritt 4: Metriken-Funktion definieren...
Metriken-Funktion definiert.

Schritt 5: Training konfigurieren und starten...
Training startet...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.217100,1.078446,0.900000,0.899749,0.904040,0.900000
2,0.807300,0.694583,0.950000,0.949875,0.954545,0.950000
3,0.635200,0.559012,0.950000,0.950957,0.958333,0.950000


Training abgeschlossen.

Schritt 6: Modell speichern in './my_german_intent_analyzer'...
Modell und Tokenizer erfolgreich gespeichert.


In [9]:
print("\nSchritt 7: Finalen Testscore abrufen...")
test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)

print("\n--- Finaler Testscore ---")
for metric_name, value in test_results.items():
    print(f"{metric_name}: {value:.4f}")

# Zusätzliche Ausgabe für richtige und falsche Vorhersagen
if 'eval_correct_predictions' in test_results and 'eval_incorrect_predictions' in test_results:
    print(f"\nRichtige Vorhersagen: {int(test_results['eval_correct_predictions'])} von {len(tokenized_test_dataset)}")
    print(f"Falsche Vorhersagen: {int(test_results['eval_incorrect_predictions'])} von {len(tokenized_test_dataset)}")


Schritt 7: Finalen Testscore abrufen...

--- Finaler Testscore ---
eval_loss: 0.5590
eval_accuracy: 0.9500
eval_f1: 0.9510
eval_precision: 0.9583
eval_recall: 0.9500
eval_runtime: 0.4376
eval_samples_per_second: 91.4120
eval_steps_per_second: 11.4270
epoch: 3.0000
